In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!git clone https://github.com/seujung/KoBART-summarization.git
!cd KoBART-summarization && ls

Cloning into 'KoBART-summarization'...
remote: Enumerating objects: 155, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 155 (delta 49), reused 44 (delta 40), pack-reused 83 (from 1)
Receiving objects: 100% (155/155), 37.24 MiB | 38.05 MiB/s, done.
Resolving deltas: 100% (79/79), done.
data		    get_model_binary.py  install_kobart.sh  README.md	      run_train.sh
dataset.py	    imgs		 LICENSE	    requirements.txt  train.py
download_binary.py  infer.py		 model.py	    rouge_metric.py


In [4]:
!pip install loguru
!pip install lightning
!pip install torchmetrics==0.6.0
!pip install pytorch_lightning==1.5.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.4/329.4 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: torchmetrics
    Found existing installation: torchmetrics 1.6.0
    Uninstalling torchmetrics-1.6.0:
      Successfully uninstalled torchmetrics-1.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lightning 2.4.0 requires torchmetrics<3.0,>=0.7.0, but you have torchmetrics 0.6.0 which is incompatible.
pytorch-lightning 2.4.0 requires torchmetrics>=0.7.0, but you have torchmetrics 0.6.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import os

# 현재 작업 디렉토리 확인
print(os.getcwd())

# 디렉토리 변경
os.chdir("/content/KoBART-summarization")
print(os.getcwd())

/content
/content/KoBART-summarization


In [7]:
import json
import csv

# JSON 파일 로드
with open("/content/sample_data/naver_news_cleaned (2).json", "r", encoding="utf-8") as f:
    data = json.load(f)

# TSV 파일로 저장
with open("train.tsv", "w", encoding="utf-8", newline="") as f:
    tsv_writer = csv.writer(f, delimiter="\t")
    # 헤더 작성
    tsv_writer.writerow(["text", "keywords"])
    # 데이터 작성
    for item in data:
        tsv_writer.writerow([item["text"], " ".join(item["keywords"])])

In [8]:
!mv train.tsv data/

In [13]:
!ls data/

test.tar.gz  train.tar.gz  train.tsv


In [10]:
!cp data/train.tsv data/test.tsv

In [11]:
import pandas as pd

# train.tsv 불러오기
train_data = pd.read_csv("data/train.tsv", sep="\t")

# 80%는 train, 20%는 test로 분리
train_split = train_data.sample(frac=0.8, random_state=42)
test_split = train_data.drop(train_split.index)

# 분리된 데이터를 저장
train_split.to_csv("data/train.tsv", sep="\t", index=False)
test_split.to_csv("data/test.tsv", sep="\t", index=False)

In [12]:
import pandas as pd

# train.tsv 헤더 수정
train_data = pd.read_csv("data/train.tsv", sep="\t")
train_data.rename(columns={"text": "news"}, inplace=True)
train_data.to_csv("data/train.tsv", sep="\t", index=False)

# test.tsv 헤더 수정
test_data = pd.read_csv("data/test.tsv", sep="\t")
test_data.rename(columns={"text": "news"}, inplace=True)
test_data.to_csv("data/test.tsv", sep="\t", index=False)

In [13]:
import pandas as pd

# train.tsv 수정
train_data = pd.read_csv("data/train.tsv", sep="\t")
train_data.rename(columns={"keywords": "summary"}, inplace=True)
train_data.to_csv("data/train.tsv", sep="\t", index=False)

# test.tsv 수정
test_data = pd.read_csv("data/test.tsv", sep="\t")
test_data.rename(columns={"keywords": "summary"}, inplace=True)
test_data.to_csv("data/test.tsv", sep="\t", index=False)

In [14]:
import pandas as pd

# 데이터 로드
train_data = pd.read_csv("data/train.tsv", sep="\t")
test_data = pd.read_csv("data/test.tsv", sep="\t")

# 데이터 검증
print("Train 데이터 Null 값:", train_data.isnull().sum())
print("Test 데이터 Null 값:", test_data.isnull().sum())

# Null 값 제거
train_data = train_data.dropna()
test_data = test_data.dropna()

# 수정된 데이터 저장
train_data.to_csv("data/train.tsv", sep="\t", index=False)
test_data.to_csv("data/test.tsv", sep="\t", index=False)

Train 데이터 Null 값: news       0
summary    1
dtype: int64
Test 데이터 Null 값: news       0
summary    0
dtype: int64


In [15]:
!WANDB_MODE=disabled python train.py \
    --train_file data/train.tsv \
    --test_file data/test.tsv \
    --batch_size 16 \
    --max_epochs 5 \
    --gradient_clip_val 1.0 \
    --num_workers 4 \
    --accelerator gpu \
    --num_gpus 1 \
    --lr 3e-5

2024-12-09 12:22:28.396233: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-09 12:22:28.419950: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-09 12:22:28.427368: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-09 12:22:28.445574: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-09 12:22:30.601026: W tensorflow/compiler/tf2

In [31]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
import torch

# KoBART 모델 및 토크나이저 로드
checkpoint_path = "/content/KoBART-summarization/checkpoint/model_chp/epoch=04-val_loss=0.882.ckpt"
model = BartForConditionalGeneration.from_pretrained("hyunwoongko/kobart")
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
model.load_state_dict(checkpoint["state_dict"], strict=False)

tokenizer = PreTrainedTokenizerFast.from_pretrained("hyunwoongko/kobart")

# 테스트 문장 요약
test_text = """
유방암 재발 걱정인데 HER2 세분화 연구로 더 정밀하게 예측 강남세브란스 연구팀 발표
유방암 환자 2천여명 분석 HER2 저발현 그룹이 음성보다 재발 확률 높아 유방암은 환자가
지닌 인자 유무에 따라 여러 아형으로 분류된다 전통적으로는 호르몬 수용체를 갖고 있는지에
따라 양성과 음성으로 나눈 뒤 그 안에서 HER2 단백질이 있는지 없는지 살폈다 HER2란
인간 표피 성장 인자 수용체 2형 단백질로 암세포 표면에 HER2가 무한 증식을 일으키면
문제가 될 수 있다 최근에는 호르몬 수용체를 지녔지만 HER2 단백질이 없는 그룹을 보다
세부적으로 분류하는 추세다 HER2 저발현 그룹 HER2Low과 HER2 음성 HER2zero으로 나누는
것이 대표적이다...
"""

# 입력 토큰 변환
inputs = tokenizer(test_text, return_tensors="pt", truncation=True, max_length=512)

# 요약 생성
summary_ids = model.generate(
    inputs["input_ids"],
    max_length=10,  # 최대 10글자
    min_length=10,  # 최소 10글자
    num_beams=6,    # 빔 서치 강화
    no_repeat_ngram_size=3,  # 반복 방지
    length_penalty=1.5,  # 요약 길이 강화
    early_stopping=True
)

# 요약 디코딩
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("10글자 요약:", summary)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
<ipython-input-31-b65b5bbb6049>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on Gi

10글자 요약: 
 
 
유방암 재발 걱정


In [ ]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
import torch

# 모델 저장 경로
model_save_path = "/content/kobart_finetuned"

# 학습한 모델 로드 (이미 학습했다고 가정)
checkpoint_path = "/content/KoBART-summarization/checkpoint/model_chp/epoch=04-val_loss=0.882.ckpt"
model = BartForConditionalGeneration.from_pretrained("hyunwoongko/kobart")
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
model.load_state_dict(checkpoint["state_dict"], strict=False)

# 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("hyunwoongko/kobart")

# 모델과 토크나이저 저장
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"Model and tokenizer saved to {model_save_path}")